In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
from scipy.integrate import dblquad
from scipy.optimize import minimize
import warnings

# Ignore warnings from integration for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# =============================================================================
# 1. DEFINE DISTRIBUTION AND CONSTANTS
# =============================================================================
# Use two different Beta distributions to create a non-symmetric joint PDF
a_x, b_x = 2, 5  # Parameters for X distribution
a_y, b_y = 4, 3  # Parameters for Y distribution

# The constraint value C_0 must be greater than the means of X and Y
C0 = 0.65

# The joint PDF f(x,y) is the product of the individual PDFs
def pdf(y, x):
    """
    Joint PDF f(x,y). Note: scipy.dblquad expects func(y, x).
    """
    return beta.pdf(x, a_x, b_x) * beta.pdf(y, a_y, b_y)

# Verify that the C0 assumption holds
mean_x = a_x / (a_x + b_x)
mean_y = a_y / (a_y + b_y)
print(f"Distribution Setup:")
print(f"  E[X] = {mean_x:.3f}")
print(f"  E[Y] = {mean_y:.3f}")
print(f"  Constraint Value C0 = {C0}\n")
assert C0 > mean_x and C0 > mean_y, "C0 must be greater than E[X] and E[Y]"

# =============================================================================
# 2. DEFINE OBJECTIVE AND CONSTRAINT FUNCTIONS
# =============================================================================

def objective_function(params):
    """
    The objective is to maximize P(M_x) + P(M_y), which is equivalent to
    minimizing P(N), the probability of the bottom-left rectangular region.
    """
    x_hat, y_hat, k = params
    # P(N) is the integral of the PDF over the rectangle [0, x_hat] x [0, y_hat]
    prob_N, _ = dblquad(pdf, 0, x_hat, lambda x: 0, lambda x: y_hat)
    return prob_N

def constraint_Mx(params):
    """
    Calculates the first constraint: E[X | M_x] - C0 = 0
    """
    x_hat, y_hat, k = params
    if k <= 0: return 1e6 # Penalize invalid k values

    # Region M_x consists of two parts: a rectangle and the area under the line in R3
    
    # Part 1: Rectangle R_1 = [x_hat, 1] x [0, y_hat]
    num1, _ = dblquad(lambda y, x: x * pdf(y, x), x_hat, 1, lambda x: 0, lambda x: y_hat)
    den1, _ = dblquad(pdf, x_hat, 1, lambda x: 0, lambda x: y_hat)

    # Part 2: Area in R3 below the line y = y_hat + k*(x-x_hat)
    # This is easier to integrate by swapping x and y order.
    # The line is x = x_hat + (y - y_hat)/k
    line_x = lambda y_val: x_hat + (y_val - y_hat) / k
    num2, _ = dblquad(lambda x, y: x * pdf(y, x), y_hat, 1, line_x, lambda y: 1)
    den2, _ = dblquad(pdf, y_hat, 1, line_x, lambda y: 1)
    
    # Total conditional expectation
    total_num = num1 + num2
    total_den = den1 + den2
    
    if total_den == 0: return 1e6 # Avoid division by zero
    
    expected_x = total_num / total_den
    return expected_x - C0

def constraint_My(params):
    """
    Calculates the second constraint: E[Y | M_y] - C0 = 0
    """
    x_hat, y_hat, k = params
    if k <= 0: return 1e6 # Penalize invalid k values

    # Region M_y consists of two parts: a rectangle and the area above the line in R3

    # Part 1: Rectangle R_2 = [0, x_hat] x [y_hat, 1]
    num1, _ = dblquad(lambda x, y: y * pdf(y, x), y_hat, 1, lambda x: 0, lambda x: x_hat)
    den1, _ = dblquad(pdf, y_hat, 1, lambda x: 0, lambda x: x_hat)

    # Part 2: Area in R3 above the line y = y_hat + k*(x-x_hat)
    line_y = lambda x_val: y_hat + k * (x_val - x_hat)
    num2, _ = dblquad(lambda y, x: y * pdf(y, x), x_hat, 1, line_y, lambda x: 1)
    den2, _ = dblquad(pdf, x_hat, 1, line_y, lambda x: 1)
    
    # Total conditional expectation
    total_num = num1 + num2
    total_den = den1 + den2

    if total_den == 0: return 1e6 # Avoid division by zero
    
    expected_y = total_num / total_den
    return expected_y - C0

# =============================================================================
# 3. RUN THE OPTIMIZATION
# =============================================================================
print("Running optimization... (this may take a minute)")

# Initial guess for [x_hat, y_hat, k]
# Start near the means, with a slope of 1
initial_guess = [C0, C0, 1.0]

# Bounds for the variables to keep them within the unit square and k positive
bounds = [(0.01, 0.99), (0.01, 0.99), (0.01, 10.0)]

# Define the constraints for the optimizer
constraints = [
    {'type': 'eq', 'fun': constraint_Mx},
    {'type': 'eq', 'fun': constraint_My}
]

# Run the SLSQP optimizer
result = minimize(
    fun=objective_function,
    x0=initial_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': True, 'ftol': 1e-7} # ftol is the precision goal
)

# =============================================================================
# 4. DISPLAY RESULTS
# =============================================================================
print("\n" + "="*50)
print("Optimization Finished")
print("="*50)

if result.success:
    x_opt, y_opt, k_opt = result.x
    print(f"Optimal solution found:")
    print(f"  x* = {x_opt:.4f}")
    print(f"  y* = {y_opt:.4f}")
    print(f"  k* = {k_opt:.4f}")
    
    # Verify constraints at the solution
    c1_val = constraint_Mx(result.x)
    c2_val = constraint_My(result.x)
    print("\nConstraint values at solution (should be close to 0):")
    print(f"  Constraint E[X|M_x] - C0 = {c1_val:.3e}")
    print(f"  Constraint E[Y|M_y] - C0 = {c2_val:.3e}")
    
    # Plotting the results
    x_grid = np.linspace(0, 1, 100)
    y_grid = np.linspace(0, 1, 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    Z = pdf(Y, X)

    plt.figure(figsize=(8, 8))
    plt.contourf(X, Y, Z, levels=15, cmap='viridis', alpha=0.7)
    plt.colorbar(label='Probability Density')
    
    # Draw the rectangle N
    rect = plt.Rectangle((0, 0), x_opt, y_opt, facecolor='red', alpha=0.5, label='Region N')
    plt.gca().add_patch(rect)
    
    # Draw the separation line
    line_x_vals = np.linspace(x_opt, 1, 100)
    line_y_vals = y_opt + k_opt * (line_x_vals - x_opt)
    plt.plot(line_x_vals, line_y_vals, 'r-', linewidth=3, label='Separating Line')
    
    # Annotate regions
    plt.text(x_opt / 2, y_opt / 2, 'N', color='white', ha='center', va='center', fontsize=15)
    plt.text(x_opt + (1-x_opt)/2, y_opt/2, '$M_x$', color='black', ha='center', va='center', fontsize=15)
    plt.text(x_opt / 2, y_opt + (1-y_opt)/2, '$M_y$', color='black', ha='center', va='center', fontsize=15)

    plt.title('Optimal Partition of the Joint Distribution')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

else:
    print("Optimization failed.")
    print(result.message)

Distribution Setup:
  E[X] = 0.286
  E[Y] = 0.571
  Constraint Value C0 = 0.65

Running optimization... (this may take a minute)
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 0.00018553013923335634
            Iterations: 31
            Function evaluations: 238
            Gradient evaluations: 27

Optimization Finished
Optimization failed.
Positive directional derivative for linesearch
